# Protein MD Setup tutorial using BioExcel Building Blocks (biobb) through REST API
**Based on the official GROMACS tutorial:** [http://www.mdtutorials.com/gmx/lysozyme/index.html](http://www.mdtutorials.com/gmx/lysozyme/index.html)
***
This tutorial aims to illustrate the process of **setting up a simulation system** containing a **protein**, step by step, using the **BioExcel Building Blocks (biobb) [REST API](https://mmb.irbbarcelona.org/biobb-api)**. The particular example used is the **Lysozyme** protein (PDB code 1AKI). 
***

## Settings
 
### Auxiliar libraries used

 - [requests](https://pypi.org/project/requests/): Requests allows you to send *organic, grass-fed* HTTP/1.1 requests, without the need for manual labor. 
 - [nb_conda_kernels](https://github.com/Anaconda-Platform/nb_conda_kernels): Enables a Jupyter Notebook or JupyterLab application in one conda environment to access kernels for Python, R, and other languages found in other environments.
 - [nglview](http://nglviewer.org/#nglview): Jupyter/IPython widget to interactively view molecular structures and trajectories in notebooks.
 - [ipywidgets](https://github.com/jupyter-widgets/ipywidgets): Interactive HTML widgets for Jupyter notebooks and the IPython kernel.
 - [plotly](https://plot.ly/python/offline/): Python interactive graphing library integrated in Jupyter notebooks.
 - [simpletraj](https://github.com/arose/simpletraj): Lightweight coordinate-only trajectory reader based on code from GROMACS, MDAnalysis and VMD.

### Conda Installation and Launch

```console
 git clone https://github.com/bioexcel/biobb_wf_md_setup_api.git
 cd biobb_wf_md_setup_api
 conda env create -f conda_env/environment.yml
 conda activate biobb_MDsetupAPI_tutorial
 jupyter-nbextension enable --py --user widgetsnbextension
 jupyter-nbextension enable --py --user nglview
 jupyter-notebook biobb_wf_md_setup_api/notebooks/biobb_MDsetupAPI_tutorial.ipynb
  ``` 

***
 
## Pipeline steps
 1. [Input Parameters](#input)
 2. [Fetching PDB Structure](#fetch)
 3. [Fix Protein Structure](#fix)
 4. [Create Protein System Topology](#top)
 5. [Create Solvent Box](#box)
 6. [Fill the Box with Water Molecules](#water)
 7. [Adding Ions](#ions)
 8. [Energetically Minimize the System](#min)
 9. [Equilibrate the System (NVT)](#nvt)
 10. [Equilibrate the System (NPT)](#npt)
 11. [Free Molecular Dynamics Simulation](#free)
 12. [Post-processing and Visualizing Resulting 3D Trajectory](#post)
 13. [Output Files](#output)
 
***
<img src="https://bioexcel.eu/wp-content/uploads/2019/04/Bioexcell_logo_1080px_transp.png" alt="Bioexcel2 logo"
	title="Bioexcel2 logo" width="400" />
***


<a id="input"></a>
## Input parameters
**Input parameters** needed:
 - **pdbCode**: PDB code of the protein structure (e.g. 1AKI)
 - **apiURL**: Base URL for the Biobb REST API (https://mmb.irbbarcelona.org/biobb-api/rest/v1/)
 
Additionally, the **utils** library is loaded. This library contains global functions that are used for sending and retrieving data to / from the REST API. [Click here](https://mmb.irbbarcelona.org/biobb-api/tutorial) for more information about how the BioBB REST API works and which is the purpose for each of these functions.

In [1]:
import nglview
import ipywidgets
from utils import *

pdbCode = "1AKI"
apiURL  = "https://mmb.irbbarcelona.org/biobb-api/rest/v1/" 

_ColormakerRegistry()

<a id="fetch"></a>
***
## Fetching PDB structure
Downloading **PDB structure** with the **protein molecule** from the RCSB PDB database.<br>
Alternatively, a **PDB file** can be used as starting structure. <br>

***
**BioBB REST API** end points used:
 - [PDB](https://mmb.irbbarcelona.org/biobb-api/rest/v1/launch/biobb_io/pdb) from **biobb_io.api.pdb**
***

In [3]:
# Downloading desired PDB file

# Create properties dict and inputs/outputs
downloaded_pdb = pdbCode + '.pdb'
prop = {
    'pdb_code': pdbCode
}

# Launch bb on REST API
token = launch_job(url = apiURL + 'launch/biobb_io/pdb', 
                   config = prop,
                   output_pdb_path = downloaded_pdb)

{
  "code": 303,
  "state": "RUNNING",
  "message": "The requested job has has been successfully launched, please go to /retrieve/status/{token} for checking job status.",
  "token": "26d905e6182af8b153f08e3f02b81a8ca212f8d57183d6da059c8560e8598b0691eba6e57dd00bbc4a1fb096277f0babe69629e619cc7934793d3d9c6d3552a5"
}


In [4]:
# Check job status
out_files = check_job(token, apiURL)

Total elapsed time: 0:00:10
REST API JSON response:
{
  "code": 200,
  "state": "FINISHED",
  "message": "The requested job has finished successfully, please go to /retrieve/data/{id} for each output_files.",
  "output_files": [
    {
      "id": "5ec24e959257d1.53480788",
      "name": "1AKI.pdb",
      "size": 81081,
      "mimetype": "text/plain"
    }
  ],
  "expiration": "May 20, 2020 00:00 GMT+0000"
}


In [5]:
# Save generated file to disk
retrieve_data(out_files, apiURL)

<a id="vis3D"></a>
### Visualizing 3D structure
Visualizing the downloaded/given **PDB structure** using **NGL**:    

In [6]:
# Show protein
view = nglview.show_structure_file(downloaded_pdb)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

<img src='ngl1.png'></img>

<a id="fix"></a>
***
## Fix protein structure
**Checking** and **fixing** (if needed) the protein structure:<br>
- **Modeling** **missing side-chain atoms**, modifying incorrect **amide assignments**, choosing **alternative locations**.<br>
- **Checking** for missing **backbone atoms**, **heteroatoms**, **modified residues** and possible **atomic clashes**.

***
**BioBB REST API** end points used:
 - [FixSideChain](https://mmb.irbbarcelona.org/biobb-api/rest/v1/launch/biobb_model/fix_side_chain) from **biobb_model.model.fix_side_chain**
***

In [7]:
# Check & Fix PDB

# Create inputs/outputs
fixed_pdb = pdbCode + '_fixed.pdb'

# Launch bb on REST API
token = launch_job(url = apiURL + 'launch/biobb_model/fix_side_chain',
                   input_pdb_path = downloaded_pdb,
                   output_pdb_path = fixed_pdb)

{
  "code": 303,
  "state": "RUNNING",
  "message": "The requested job has has been successfully launched, please go to /retrieve/status/{token} for checking job status.",
  "token": "04e582f8c528415d53939e00d2280bd06e5733fbebacf90c60ab0b695cb224faa3d9a6fa459b1d5dc1ec3a29e42f57bc3de3a10ade140fb6f6ee7a56a1dde2b0"
}


In [8]:
# Check job status
out_files = check_job(token, apiURL)

Total elapsed time: 0:00:01
REST API JSON response:
{
  "code": 200,
  "state": "FINISHED",
  "message": "The requested job has finished successfully, please go to /retrieve/data/{id} for each output_files.",
  "output_files": [
    {
      "id": "5ec24ed1435c71.10854641",
      "name": "1AKI_fixed.pdb",
      "size": 81167,
      "mimetype": "text/plain"
    }
  ],
  "expiration": "May 20, 2020 00:00 GMT+0000"
}


In [9]:
# Save generated file to disk
retrieve_data(out_files, apiURL)

### Visualizing 3D structure
Visualizing the fixed **PDB structure** using **NGL**. In this particular example, the checking step didn't find any issue to be solved, so there is no difference between the original structure and the fixed one.   

In [10]:
# Show protein
view = nglview.show_structure_file(fixed_pdb)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

<img src='ngl2.png'></img>

<a id="top"></a>
***
## Create protein system topology
**Building GROMACS topology** corresponding to the protein structure.<br>
Force field used in this tutorial is [**amber99sb-ildn**](https://dx.doi.org/10.1002%2Fprot.22711): AMBER **parm99** force field with **corrections on backbone** (sb) and **side-chain torsion potentials** (ildn). Water molecules type used in this tutorial is [**spc/e**](https://pubs.acs.org/doi/abs/10.1021/j100308a038).<br>
Adding **hydrogen atoms** if missing. Automatically identifying **disulfide bridges**. <br>

Generating two output files: 
- **GROMACS structure** (gro file)
- **GROMACS topology** ZIP compressed file containing:
    - *GROMACS topology top file* (top file)
    - *GROMACS position restraint file/s* (itp file/s)
***
**BioBB REST API** end points used:
 - [Pdb2gmx](https://mmb.irbbarcelona.org/biobb-api/rest/v1/launch/biobb_md/pdb2gmx) from **biobb_md.gromacs.pdb2gmx**
***

In [11]:
# Create system topology

# Create inputs/outputs
output_pdb2gmx_gro = pdbCode + '_pdb2gmx.gro'
output_pdb2gmx_top_zip = pdbCode + '_pdb2gmx_top.zip'

# Launch bb on REST API
token = launch_job(url = apiURL + 'launch/biobb_md/pdb2gmx', 
                   input_pdb_path = fixed_pdb,
                   output_gro_path = output_pdb2gmx_gro,
                   output_top_zip_path = output_pdb2gmx_top_zip)

{
  "code": 303,
  "state": "RUNNING",
  "message": "The requested job has has been successfully launched, please go to /retrieve/status/{token} for checking job status.",
  "token": "89e61ca8c7adfe269253710ac732f93f5d17f9c1a2c6056bb17b6083475d44b6cee35611b84c0057bfc1f447275edfa70b2de71416767c84207f2d060b6c7228"
}


In [12]:
# Check job status
out_files = check_job(token, apiURL)

Total elapsed time: 0:00:09
REST API JSON response:
{
  "code": 200,
  "state": "FINISHED",
  "message": "The requested job has finished successfully, please go to /retrieve/data/{id} for each output_files.",
  "output_files": [
    {
      "id": "5ec24eef5933d6.64638162",
      "name": "1AKI_pdb2gmx.gro",
      "size": 88286,
      "mimetype": "application/octet-stream"
    },
    {
      "id": "5ec24eef5b85a5.82321574",
      "name": "1AKI_pdb2gmx_top.zip",
      "size": 588585,
      "mimetype": "application/zip"
    }
  ],
  "expiration": "May 20, 2020 00:00 GMT+0000"
}


In [13]:
# Save generated file to disk
retrieve_data(out_files, apiURL)

### Visualizing 3D structure
Visualizing the generated **GRO structure** using **NGL**. Note that **hydrogen atoms** were added to the structure by the **pdb2gmx GROMACS tool** when generating the **topology**.    

In [14]:
# Show protein
view = nglview.show_structure_file(output_pdb2gmx_gro)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

<img src='ngl3.png'></img>

<a id="box"></a>
***
## Create solvent box
Define the unit cell for the **protein structure MD system** to fill it with water molecules.<br>
A **cubic box** is used to define the unit cell, with a **distance from the protein to the box edge of 1.0 nm**. The protein is **centered in the box**.  

***
**BioBB REST API** end points used:
 - [Editconf](https://mmb.irbbarcelona.org/biobb-api/rest/v1/launch/biobb_md/editconf) from **biobb_md.gromacs.editconf**
***

In [15]:
# Editconf: Create solvent box

# Create properties dict and inputs/outputs
output_editconf_gro = pdbCode + '_editconf.gro'
prop = {
    'box_type': 'cubic',
    'distance_to_molecule': 1.0
}

# Launch bb on REST API
token = launch_job(url = apiURL + 'launch/biobb_md/editconf', 
                   config = prop,
                   input_gro_path = output_pdb2gmx_gro,
                   output_gro_path = output_editconf_gro)

{
  "code": 303,
  "state": "RUNNING",
  "message": "The requested job has has been successfully launched, please go to /retrieve/status/{token} for checking job status.",
  "token": "a404ba1f8fa97c6d712080a503edf152d4973616e77fbd4cd4e6f496d428ba58971e75521d89bfd6436a658b4f734fec317dc4bf058c0153bd97d400e4e927b2"
}


In [16]:
# Check job status
out_files = check_job(token, apiURL)

Total elapsed time: 0:00:01
REST API JSON response:
{
  "code": 200,
  "state": "FINISHED",
  "message": "The requested job has finished successfully, please go to /retrieve/data/{id} for each output_files.",
  "output_files": [
    {
      "id": "5ec24efdbf8120.57722845",
      "name": "1AKI_editconf.gro",
      "size": 88286,
      "mimetype": "application/octet-stream"
    }
  ],
  "expiration": "May 20, 2020 00:00 GMT+0000"
}


In [17]:
# Save generated file to disk
retrieve_data(out_files, apiURL)

<a id="water"></a>
***
## Fill the box with water molecules
Fill the unit cell for the **protein structure system** with water molecules.<br>
The solvent type used is the default **Simple Point Charge water (SPC)**, a generic equilibrated 3-point solvent model. 

***
**BioBB REST API** end points used:
 - [Solvate](https://mmb.irbbarcelona.org/biobb-api/rest/v1/launch/biobb_md/solvate) from **biobb_md.gromacs.solvate**
***

In [18]:
# Solvate: Fill the box with water molecules

# Create inputs/outputs
output_solvate_gro = pdbCode + '_solvate.gro'
output_solvate_top_zip = pdbCode + '_solvate_top.zip'

# Launch bb on REST API
token = launch_job(url = apiURL + 'launch/biobb_md/solvate', 
                   input_solute_gro_path = output_editconf_gro,
                   input_top_zip_path = output_pdb2gmx_top_zip,
                   output_gro_path = output_solvate_gro,
                   output_top_zip_path = output_solvate_top_zip)

{
  "code": 303,
  "state": "RUNNING",
  "message": "The requested job has has been successfully launched, please go to /retrieve/status/{token} for checking job status.",
  "token": "eaeb9bdd8cf4b1bf2455d3717cdeab9a809b1761760f101f30e9101c93694ed260f724d5eee1f6894bc0c99e5b643f773caf08104c4a49e4699c04015e68ec4c"
}


In [19]:
# Check job status
out_files = check_job(token, apiURL)

Total elapsed time: 0:00:05
REST API JSON response:
{
  "code": 200,
  "state": "FINISHED",
  "message": "The requested job has finished successfully, please go to /retrieve/data/{id} for each output_files.",
  "output_files": [
    {
      "id": "5ec24f0d843a77.61157992",
      "name": "1AKI_solvate.gro",
      "size": 1525226,
      "mimetype": "application/octet-stream"
    },
    {
      "id": "5ec24f0d9959b2.24701418",
      "name": "1AKI_solvate_top.zip",
      "size": 588616,
      "mimetype": "application/zip"
    }
  ],
  "expiration": "May 20, 2020 00:00 GMT+0000"
}


In [20]:
# Save generated file to disk
retrieve_data(out_files, apiURL)

### Visualizing 3D structure
Visualizing the **protein system** with the newly added **solvent box** using **NGL**.<br> Note the **cubic box** filled with **water molecules** surrounding the **protein structure**, which is **centered** right in the middle of the cube.

In [21]:
# Show protein
view = nglview.show_structure_file(output_solvate_gro)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='solute', color='green')
view.add_representation(repr_type='ball+stick', selection='SOL')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

<img src='ngl4.png'></img>

<a id="ions"></a>
***
## Adding ions
Add ions to neutralize the **protein structure** charge
- [Step 1](#ionsStep1): Creating portable binary run file for ion generation
- [Step 2](#ionsStep2): Adding ions to **neutralize** the system

***
**BioBB REST API** end points used:
 - [Grompp](https://mmb.irbbarcelona.org/biobb-api/rest/v1/launch/biobb_md/grompp) from **biobb_md.gromacs.grompp**
 - [Genion](https://mmb.irbbarcelona.org/biobb-api/rest/v1/launch/biobb_md/genion) from **biobb_md.gromacs.genion**
***

<a id="ionsStep1"></a>
### Step 1: Creating portable binary run file for ion generation
A simple **energy minimization** molecular dynamics parameters (mdp) properties will be used to generate the portable binary run file for **ion generation**, although **any legitimate combination of parameters** could be used in this step.

In [22]:
# Grompp: Creating portable binary run file for ion generation

# Create prop dict and inputs/outputs
output_gppion_tpr = pdbCode + '_gppion.tpr'
prop = {
    'mdp':{
        'type': 'minimization'
    }
}

# Launch bb on REST API
token = launch_job(url = apiURL + 'launch/biobb_md/grompp', 
                   config = prop,
                   input_gro_path = output_solvate_gro,
                   input_top_zip_path = output_solvate_top_zip,
                   output_tpr_path = output_gppion_tpr)

{
  "code": 303,
  "state": "RUNNING",
  "message": "The requested job has has been successfully launched, please go to /retrieve/status/{token} for checking job status.",
  "token": "50bac270a1a61d3206a5b6f27ef9a85f9f4477aa349e3a5645f6fb8fab0cf37490548e4602659db1ce70e943f560ae309a3c2faefce0c6f8a0ed7ca42e980323"
}


In [23]:
# Check job status
out_files = check_job(token, apiURL)

Total elapsed time: 0:00:02
REST API JSON response:
{
  "code": 200,
  "state": "FINISHED",
  "message": "The requested job has finished successfully, please go to /retrieve/data/{id} for each output_files.",
  "output_files": [
    {
      "id": "5ec24f1c678f09.30090542",
      "name": "1AKI_gppion.tpr",
      "size": 1398340,
      "mimetype": "application/octet-stream"
    }
  ],
  "expiration": "May 20, 2020 00:00 GMT+0000"
}


In [24]:
# Save generated file to disk
retrieve_data(out_files, apiURL)

<a id="ionsStep2"></a>
### Step 2: Adding ions to neutralize the system
Replace **solvent molecules** with **ions** to **neutralize** the system.

In [25]:
# Genion: Adding ions to neutralize the system

# Create prop dict and inputs/outputs
output_genion_gro = pdbCode + '_genion.gro'
output_genion_top_zip = pdbCode + '_genion_top.zip'
prop={
    'neutral':True
}

# Launch bb on REST API
token = launch_job(url = apiURL + 'launch/biobb_md/genion',
                   config = prop,
                   input_tpr_path = output_gppion_tpr,
                   input_top_zip_path = output_solvate_top_zip,
                   output_gro_path = output_genion_gro,
                   output_top_zip_path = output_genion_top_zip)

{
  "code": 303,
  "state": "RUNNING",
  "message": "The requested job has has been successfully launched, please go to /retrieve/status/{token} for checking job status.",
  "token": "d7c85a995446b984f4e8a419d509aa27cd05d80c9ab7dd6772d4228690376ca09c9c35d4330a3d9f9476f1526d38ee42a80635ce72f340facaf4f366e1db8c40"
}


In [26]:
# Check job status
out_files = check_job(token, apiURL)

Total elapsed time: 0:00:02
REST API JSON response:
{
  "code": 200,
  "state": "FINISHED",
  "message": "The requested job has finished successfully, please go to /retrieve/data/{id} for each output_files.",
  "output_files": [
    {
      "id": "5ec24f2b12f1b1.73665989",
      "name": "1AKI_genion.gro",
      "size": 1522674,
      "mimetype": "application/octet-stream"
    },
    {
      "id": "5ec24f2b27c584.91634084",
      "name": "1AKI_genion_top.zip",
      "size": 588652,
      "mimetype": "application/zip"
    }
  ],
  "expiration": "May 20, 2020 00:00 GMT+0000"
}


In [27]:
# Save generated file to disk
retrieve_data(out_files, apiURL)

### Visualizing 3D structure
Visualizing the **neutralized protein system** with the newly added **ions** using **NGL**

In [28]:
# Show protein
view = nglview.show_structure_file(output_genion_gro)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='solute', color='sstruc')
view.add_representation(repr_type='ball+stick', selection='NA')
view.add_representation(repr_type='ball+stick', selection='CL')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

<img src='ngl5.png'></img>

<a id="min"></a>
***
## Energetically minimize the system
Energetically minimize the **protein system** till reaching a desired potential energy.
- [Step 1](#emStep1): Creating portable binary run file for energy minimization
- [Step 2](#emStep2): Energetically minimize the **system** till reaching a force of 500 kJ mol-1 nm-1.
- [Step 3](#emStep3): Checking **energy minimization** results. Plotting energy by time during the **minimization** process.

***
**BioBB REST API** end points used:
 - [Grompp](https://mmb.irbbarcelona.org/biobb-api/rest/v1/launch/biobb_md/grompp) from **biobb_md.gromacs.grompp**
 - [Mdrun](https://mmb.irbbarcelona.org/biobb-api/rest/v1/launch/biobb_md/mdrun) from **biobb_md.gromacs.mdrun**
 - [GMXEnergy](https://mmb.irbbarcelona.org/biobb-api/rest/v1/launch/biobb_analysis/gmx_energy) from **biobb_analysis.gromacs.gmx_energy**
***

<a id="emStep1"></a>
### Step 1: Creating portable binary run file for energy minimization
The **minimization** type of the **molecular dynamics parameters (mdp) property** contains the main default parameters to run an **energy minimization**:

-  integrator  = steep ; Algorithm (steep = steepest descent minimization)
-  emtol       = 1000.0 ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
-  emstep      = 0.01 ; Minimization step size (nm)
-  nsteps      = 50000 ; Maximum number of (minimization) steps to perform

In this particular example, the method used to run the **energy minimization** is the default **steepest descent**, but the **maximum force** is placed at **500 KJ/mol\*nm^2**, and the **maximum number of steps** to perform (if the maximum force is not reached) to **5,000 steps**. 

In [29]:
# Grompp: Creating portable binary run file for mdrun

# Create prop dict and inputs/outputs
output_gppmin_tpr = pdbCode + '_gppmin.tpr'
prop = {
    'mdp':{
        'type': 'minimization',
        'emtol':'500',
        'nsteps':'5000'
    }
}

# Launch bb on REST API
token = launch_job(url = apiURL + 'launch/biobb_md/grompp', 
                   config = prop,
                   input_gro_path = output_genion_gro,
                   input_top_zip_path = output_genion_top_zip,
                   output_tpr_path = output_gppmin_tpr)

{
  "code": 303,
  "state": "RUNNING",
  "message": "The requested job has has been successfully launched, please go to /retrieve/status/{token} for checking job status.",
  "token": "e67943940724ca9950697c723ce658c47fc2202358a4f0cd461cac08cab7d89f2c52d1235ed27b3aa5e6fde521d50a641e81b092bcf589ee72ad1a9a7c847f7a"
}


In [30]:
# Check job status
out_files = check_job(token, apiURL)

Total elapsed time: 0:00:01
REST API JSON response:
{
  "code": 200,
  "state": "FINISHED",
  "message": "The requested job has finished successfully, please go to /retrieve/data/{id} for each output_files.",
  "output_files": [
    {
      "id": "5ec24f3a6813e9.23267583",
      "name": "1AKI_gppmin.tpr",
      "size": 1398800,
      "mimetype": "application/octet-stream"
    }
  ],
  "expiration": "May 20, 2020 00:00 GMT+0000"
}


In [31]:
# Save generated file to disk
retrieve_data(out_files, apiURL)

<a id="emStep2"></a>
### Step 2: Running Energy Minimization
Running **energy minimization** using the **tpr file** generated in the previous step. 

In [32]:
# Mdrun: Running minimization

# Create inputs/outputs
output_min_trr = pdbCode + '_min.trr'
output_min_gro = pdbCode + '_min.gro'
output_min_edr = pdbCode + '_min.edr'
output_min_log = pdbCode + '_min.log'

# Launch bb on REST API
token = launch_job(url = apiURL + 'launch/biobb_md/mdrun', 
                   input_tpr_path = output_gppmin_tpr,
                   output_trr_path = output_min_trr,
                   output_gro_path = output_min_gro,
                   output_edr_path = output_min_edr,
                   output_log_path = output_min_log)

{
  "code": 303,
  "state": "RUNNING",
  "message": "The requested job has has been successfully launched, please go to /retrieve/status/{token} for checking job status.",
  "token": "6b80e713c466c5a75e0d71284486cf4b5596b27c5c4f469ded1695ddec7b31f3a6a81ff6e1fbe477523ec9881107a23ce485436ff01678cc83673e3e0929f3ad"
}


In [33]:
# Check job status
out_files = check_job(token, apiURL)

Total elapsed time: 0:02:00
REST API JSON response:
{
  "code": 200,
  "state": "FINISHED",
  "message": "The requested job has finished successfully, please go to /retrieve/data/{id} for each output_files.",
  "output_files": [
    {
      "id": "5ec24f8484d147.13412117",
      "name": "1AKI_min.trr",
      "size": 406152,
      "mimetype": "application/octet-stream"
    },
    {
      "id": "5ec24f848b4af8.20337739",
      "name": "1AKI_min.gro",
      "size": 1522674,
      "mimetype": "application/octet-stream"
    },
    {
      "id": "5ec24f84a04e87.41998717",
      "name": "1AKI_min.edr",
      "size": 402176,
      "mimetype": "application/octet-stream"
    },
    {
      "id": "5ec24f84a91524.82230443",
      "name": "1AKI_min.log",
      "size": 953627,
      "mimetype": "text/plain"
    }
  ],
  "expiration": "May 20, 2020 00:00 GMT+0000"
}


In [34]:
# Save generated file to disk
retrieve_data(out_files, apiURL)

<a id="emStep3"></a>
### Step 3: Checking Energy Minimization results
Checking **energy minimization** results. Plotting **potential energy** by time during the minimization process. 

In [35]:
# GMXEnergy: Getting system energy by time 

# Create prop dict and inputs/outputs
output_min_ene_xvg = pdbCode + '_min_ene.xvg'
prop = {
    'terms':  ["Potential"]
}

# Launch bb on REST API
token = launch_job(url = apiURL + 'launch/biobb_analysis/gmx_energy',
                   config = prop,
                   input_energy_path = output_min_edr,
                   output_xvg_path = output_min_ene_xvg)

{
  "code": 303,
  "state": "RUNNING",
  "message": "The requested job has has been successfully launched, please go to /retrieve/status/{token} for checking job status.",
  "token": "e7f9cd41c4cd9f0f66ba1cf09e48ff992f743fa30ca686cb44d091ca094f1685a05fba3de75832826f3f1ba8ec1f04cffdf84fd1425192d1157cc81130ca1f3a"
}


In [36]:
# Check job status
out_files = check_job(token, apiURL)

Total elapsed time: 0:00:03
REST API JSON response:
{
  "code": 200,
  "state": "FINISHED",
  "message": "The requested job has finished successfully, please go to /retrieve/data/{id} for each output_files.",
  "output_files": [
    {
      "id": "5ec24fdeb83a41.81430993",
      "name": "1AKI_min_ene.xvg",
      "size": 58203,
      "mimetype": "text/plain"
    }
  ],
  "expiration": "May 20, 2020 00:00 GMT+0000"
}


In [37]:
# Save generated file to disk
retrieve_data(out_files, apiURL)

In [38]:
import plotly
import plotly.graph_objs as go

#Read data from file and filter energy values higher than 1000 Kj/mol^-1
with open(output_min_ene_xvg,'r') as energy_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in energy_file 
            if not line.startswith(("#","@")) 
            if float(line.split()[1]) < 1000 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Energy Minimization",
                        xaxis=dict(title = "Energy Minimization Step"),
                        yaxis=dict(title = "Potential Energy KJ/mol-1")
                       )
}

plotly.offline.iplot(fig)

<a id="nvt"></a>
***
## Equilibrate the system (NVT)
Equilibrate the **protein system** in **NVT ensemble** (constant Number of particles, Volume and Temperature). Protein **heavy atoms** will be restrained using position restraining forces: movement is permitted, but only after overcoming a substantial energy penalty. The utility of position restraints is that they allow us to equilibrate our solvent around our protein, without the added variable of structural changes in the protein.

- [Step 1](#eqNVTStep1): Creating portable binary run file for system equilibration
- [Step 2](#eqNVTStep2): Equilibrate the **protein system** with **NVT** ensemble.
- [Step 3](#eqNVTStep3): Checking **NVT Equilibration** results. Plotting **system temperature** by time during the **NVT equilibration** process. 

***
**BioBB REST API** end points used:
 - [Grompp](https://mmb.irbbarcelona.org/biobb-api/rest/v1/launch/biobb_md/grompp) from **biobb_md.gromacs.grompp**
 - [Mdrun](https://mmb.irbbarcelona.org/biobb-api/rest/v1/launch/biobb_md/mdrun) from **biobb_md.gromacs.mdrun**
 - [GMXEnergy](https://mmb.irbbarcelona.org/biobb-api/rest/v1/launch/biobb_analysis/gmx_energy) from **biobb_analysis.gromacs.gmx_energy**
***

<a id="eqNVTStep1"></a>
### Step 1: Creating portable binary run file for system equilibration (NVT)
The **nvt** type of the **molecular dynamics parameters (mdp) property** contains the main default parameters to run an **NVT equilibration** with **protein restraints** (see [GROMACS mdp options](http://manual.gromacs.org/documentation/2018/user-guide/mdp-options.html)):

-  Define                   = -DPOSRES
-  integrator               = md
-  dt                       = 0.002
-  nsteps                   = 5000
-  pcoupl                   = no
-  gen_vel                  = yes
-  gen_temp                 = 300
-  gen_seed                 = -1

In this particular example, the default parameters will be used: **md** integrator algorithm, a **step size** of **2fs**, **5,000 equilibration steps** with the protein **heavy atoms restrained**, and a temperature of **300K**.

*Please note that for the sake of time this tutorial is only running 10ps of NVT equilibration, whereas in the [original example](http://www.mdtutorials.com/gmx/lysozyme/06_equil.html) the simulated time was 100ps.*

In [39]:
# Grompp: Creating portable binary run file for NVT Equilibration

# Create prop dict and inputs/outputs
output_gppnvt_tpr = pdbCode + '_gppnvt.tpr'
prop = {
    'mdp':{
        'type': 'nvt',
        'nsteps': 5000,
        'dt': 0.002,
        'define': '-DPOSRES',
        #'tc_grps': "DNA Water_and_ions" # NOTE: uncomment this line if working with DNA
    }
}

# Launch bb on REST API
token = launch_job(url = apiURL + 'launch/biobb_md/grompp',
                   config = prop,
                   input_gro_path = output_min_gro,
                   input_top_zip_path = output_genion_top_zip,
                   output_tpr_path = output_gppnvt_tpr)

{
  "code": 303,
  "state": "RUNNING",
  "message": "The requested job has has been successfully launched, please go to /retrieve/status/{token} for checking job status.",
  "token": "bdee772b7e06fc121619eeaa4344531ca7ebc763fb51b86fbc5bc2e16f5127ee8e3f3e123797bdccb4a8d179a70e5e6fb8220a3536431e611037faf166f15f40"
}


In [40]:
# Check job status
out_files = check_job(token, apiURL)

Total elapsed time: 0:00:20
REST API JSON response:
{
  "code": 200,
  "state": "FINISHED",
  "message": "The requested job has finished successfully, please go to /retrieve/data/{id} for each output_files.",
  "output_files": [
    {
      "id": "5ec24ffd83ce53.75174163",
      "name": "1AKI_gppnvt.tpr",
      "size": 1533904,
      "mimetype": "application/octet-stream"
    }
  ],
  "expiration": "May 20, 2020 00:00 GMT+0000"
}


In [41]:
# Save generated file to disk
retrieve_data(out_files, apiURL)

<a id="eqNVTStep2"></a>
### Step 2: Running NVT equilibration

Running **energy minimization** using the **tpr file** generated in the previous step.

In [42]:
# Mdrun: Running Equilibration NVT

# Create inputs/outputs
output_nvt_trr = pdbCode + '_nvt.trr'
output_nvt_gro = pdbCode + '_nvt.gro'
output_nvt_edr = pdbCode + '_nvt.edr'
output_nvt_log = pdbCode + '_nvt.log'
output_nvt_cpt = pdbCode + '_nvt.cpt'

# Launch bb on REST API
token = launch_job(url = apiURL + 'launch/biobb_md/mdrun',
                   input_tpr_path = output_gppnvt_tpr,
                   output_trr_path = output_nvt_trr,
                   output_gro_path = output_nvt_gro,
                   output_edr_path = output_nvt_edr,
                   output_log_path = output_nvt_log,
                   output_cpt_path = output_nvt_cpt)

{
  "code": 303,
  "state": "RUNNING",
  "message": "The requested job has has been successfully launched, please go to /retrieve/status/{token} for checking job status.",
  "token": "2a0c48da8fb4e0bdec3536049dbbb1f309d74166263bfb646c832abdbed1b272d4fa7d60e84657bdfc5d584de2b5561b6e17107bd7d45782bc04e39e623b0a92"
}


In [43]:
# Check job status
out_files = check_job(token, apiURL)

Total elapsed time: 0:01:00
REST API JSON response:
{
  "code": 200,
  "state": "FINISHED",
  "message": "The requested job has finished successfully, please go to /retrieve/data/{id} for each output_files.",
  "output_files": [
    {
      "id": "5ec250501233a5.64986437",
      "name": "1AKI_nvt.trr",
      "size": 8934024,
      "mimetype": "application/octet-stream"
    },
    {
      "id": "5ec25050648fe7.07083267",
      "name": "1AKI_nvt.gro",
      "size": 2334738,
      "mimetype": "application/octet-stream"
    },
    {
      "id": "5ec25050832d32.01509768",
      "name": "1AKI_nvt.edr",
      "size": 6892,
      "mimetype": "application/octet-stream"
    },
    {
      "id": "5ec2505084a3a2.83080474",
      "name": "1AKI_nvt.log",
      "size": 28690,
      "mimetype": "text/plain"
    },
    {
      "id": "5ec25050860a14.53543224",
      "name": "1AKI_nvt.cpt",
      "size": 813804,
      "mimetype": "application/octet-stream"
    }
  ],
  "expiration": "May 20, 2020 00:00 G

In [44]:
# Save generated file to disk
retrieve_data(out_files, apiURL)

<a id="eqNVTStep3"></a>
### Step 3: Checking NVT Equilibration results
Checking **NVT Equilibration** results. Plotting **system temperature** by time during the NVT equilibration process. 

In [45]:
# GMXEnergy: Getting system temperature by time during NVT Equilibration 

# Create prop dict and inputs/outputs
output_nvt_temp_xvg = pdbCode + '_nvt_temp.xvg'
prop = {
    'terms':  ["Temperature"]
}

# Launch bb on REST API
token = launch_job(url = apiURL + 'launch/biobb_analysis/gmx_energy', 
                   config = prop,
                   input_energy_path = output_nvt_edr,
                   output_xvg_path = output_nvt_temp_xvg)

{
  "code": 303,
  "state": "RUNNING",
  "message": "The requested job has has been successfully launched, please go to /retrieve/status/{token} for checking job status.",
  "token": "b30ab2bb6e29e9d5c86c81a3e2eb83cad577e4c9b15d3e774e11952e5292739845f61d2d5b9a9748864a54640ac56c3645ba032e527c3257cb6c5b1369bad294"
}


In [46]:
# Check job status
out_files = check_job(token, apiURL)

Total elapsed time: 0:00:01
REST API JSON response:
{
  "code": 200,
  "state": "FINISHED",
  "message": "The requested job has finished successfully, please go to /retrieve/data/{id} for each output_files.",
  "output_files": [
    {
      "id": "5ec25065b3ef01.57109466",
      "name": "1AKI_nvt_temp.xvg",
      "size": 275,
      "mimetype": "text/plain"
    }
  ],
  "expiration": "May 20, 2020 00:00 GMT+0000"
}


In [47]:
# Save generated file to disk
retrieve_data(out_files, apiURL)

In [48]:
import plotly
import plotly.graph_objs as go

# Read temperature data from file 
with open(output_nvt_temp_xvg,'r') as temperature_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in temperature_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Temperature during NVT Equilibration",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "Temperature (K)")
                       )
}

plotly.offline.iplot(fig)

<a id="npt"></a>
***
## Equilibrate the system (NPT)
Equilibrate the **protein system** in **NPT** ensemble (constant Number of particles, Pressure and Temperature).
- [Step 1](#eqNPTStep1): Creating portable binary run file for system equilibration
- [Step 2](#eqNPTStep2): Equilibrate the **protein system** with **NPT** ensemble.
- [Step 3](#eqNPTStep3): Checking **NPT Equilibration** results. Plotting **system pressure and density** by time during the **NPT equilibration** process.
***
**BioBB REST API** end points used:
 - [Grompp](https://mmb.irbbarcelona.org/biobb-api/rest/v1/launch/biobb_md/grompp) from **biobb_md.gromacs.grompp**
 - [Mdrun](https://mmb.irbbarcelona.org/biobb-api/rest/v1/launch/biobb_md/mdrun) from **biobb_md.gromacs.mdrun**
 - [GMXEnergy](https://mmb.irbbarcelona.org/biobb-api/rest/v1/launch/biobb_analysis/gmx_energy) from **biobb_analysis.gromacs.gmx_energy**
***

<a id="eqNPTStep1"></a>
### Step 1: Creating portable binary run file for system equilibration (NPT)

The **npt** type of the **molecular dynamics parameters (mdp) property** contains the main default parameters to run an **NPT equilibration** with **protein restraints** (see [GROMACS mdp options](http://manual.gromacs.org/documentation/2018/user-guide/mdp-options.html)):

-  Define                   = -DPOSRES
-  integrator               = md
-  dt                       = 0.002
-  nsteps                   = 5000
-  pcoupl = Parrinello-Rahman
-  pcoupltype = isotropic
-  tau_p = 1.0
-  ref_p = 1.0
-  compressibility = 4.5e-5
-  refcoord_scaling = com
-  gen_vel = no

In this particular example, the default parameters will be used: **md** integrator algorithm, a **time step** of **2fs**, **5,000 equilibration steps** with the protein **heavy atoms restrained**, and a Parrinello-Rahman **pressure coupling** algorithm.

*Please note that for the sake of time this tutorial is only running 10ps of NPT equilibration, whereas in the [original example](http://www.mdtutorials.com/gmx/lysozyme/07_equil2.html) the simulated time was 100ps.*

In [49]:
# Grompp: Creating portable binary run file for NPT System Equilibration

# Create prop dict and inputs/outputs
output_gppnpt_tpr = pdbCode + '_gppnpt.tpr'
prop = {
    'mdp':{
        'type': 'npt',
        'nsteps':'5000',
        #'tc_grps': "DNA Water_and_ions" # NOTE: uncomment this line if working with DNA
    }
}

# Launch bb on REST API
token = launch_job(url = apiURL + 'launch/biobb_md/grompp', 
                   config = prop,
                   input_gro_path = output_nvt_gro,
                   input_top_zip_path = output_genion_top_zip,
                   output_tpr_path = output_gppnpt_tpr)

{
  "code": 303,
  "state": "RUNNING",
  "message": "The requested job has has been successfully launched, please go to /retrieve/status/{token} for checking job status.",
  "token": "2b06993edf8e37c6e604c98ab01d72654a32b8d05ee7241df7f80c34b1eb56a341b6424fd7ffcb602bb840eaac8abb95ad9dacb87eaffeec31dc1f9f5667bcc0"
}


In [50]:
# Check job status
out_files = check_job(token, apiURL)

Total elapsed time: 0:00:03
REST API JSON response:
{
  "code": 200,
  "state": "FINISHED",
  "message": "The requested job has finished successfully, please go to /retrieve/data/{id} for each output_files.",
  "output_files": [
    {
      "id": "5ec250759e39c3.92157988",
      "name": "1AKI_gppnpt.tpr",
      "size": 1533904,
      "mimetype": "application/octet-stream"
    }
  ],
  "expiration": "May 20, 2020 00:00 GMT+0000"
}


In [51]:
# Save generated file to disk
retrieve_data(out_files, apiURL)

<a id="eqNPTStep2"></a>
### Step 2: Running NPT equilibration

In [52]:
# Mdrun: Running NPT System Equilibration

# Create inputs/outputs
output_npt_trr = pdbCode + '_npt.trr'
output_npt_gro = pdbCode + '_npt.gro'
output_npt_edr = pdbCode + '_npt.edr'
output_npt_log = pdbCode + '_npt.log'
output_npt_cpt = pdbCode + '_npt.cpt'

# Launch bb on REST API
token = launch_job(url = apiURL + 'launch/biobb_md/mdrun',
                   input_tpr_path = output_gppnpt_tpr,
                   output_trr_path = output_npt_trr,
                   output_gro_path = output_npt_gro,
                   output_edr_path = output_npt_edr,
                   output_log_path = output_npt_log,
                   output_cpt_path = output_npt_cpt)

{
  "code": 303,
  "state": "RUNNING",
  "message": "The requested job has has been successfully launched, please go to /retrieve/status/{token} for checking job status.",
  "token": "9c5724eb39443514c3ab00898b934561ba602908c1d6d42f657bfcc558cee463062542bc4a01b93aa598b1896b63059b1af5e19d00186bbf5c9801d3d388a0ad"
}


In [53]:
# Check job status
out_files = check_job(token, apiURL)

Total elapsed time: 0:01:00
REST API JSON response:
{
  "code": 200,
  "state": "FINISHED",
  "message": "The requested job has finished successfully, please go to /retrieve/data/{id} for each output_files.",
  "output_files": [
    {
      "id": "5ec250bb5027b6.46035306",
      "name": "1AKI_npt.trr",
      "size": 8934024,
      "mimetype": "application/octet-stream"
    },
    {
      "id": "5ec250bba21691.68026412",
      "name": "1AKI_npt.gro",
      "size": 2334738,
      "mimetype": "application/octet-stream"
    },
    {
      "id": "5ec250bbc38ad2.34036729",
      "name": "1AKI_npt.edr",
      "size": 8364,
      "mimetype": "application/octet-stream"
    },
    {
      "id": "5ec250bbc4e942.64066383",
      "name": "1AKI_npt.log",
      "size": 28584,
      "mimetype": "text/plain"
    },
    {
      "id": "5ec250bbc64b73.80218693",
      "name": "1AKI_npt.cpt",
      "size": 814176,
      "mimetype": "application/octet-stream"
    }
  ],
  "expiration": "May 20, 2020 00:00 G

In [54]:
# Save generated file to disk
retrieve_data(out_files, apiURL)

<a id="eqNPTStep3"></a>
### Step 3: Checking NPT Equilibration results
Checking **NPT Equilibration** results. Plotting **system pressure and density** by time during the **NPT equilibration** process. 

In [55]:
# GMXEnergy: Getting system pressure and density by time during NPT Equilibration 

# Create prop dict and inputs/outputs
output_npt_pd_xvg = pdbCode + '_npt_PD.xvg'
prop = {
    'terms':  ["Pressure","Density"]
}

# Launch bb on REST API
token = launch_job(url = apiURL + 'launch/biobb_analysis/gmx_energy',
                   config = prop,
                   input_energy_path = output_npt_edr,
                   output_xvg_path = output_npt_pd_xvg)

{
  "code": 303,
  "state": "RUNNING",
  "message": "The requested job has has been successfully launched, please go to /retrieve/status/{token} for checking job status.",
  "token": "9309e382ca3e1e3142aa4ff8a855a487910cddeaf9d738a05031bc026b552d42c9852f24d9f1d15499fe3adc3274ce45f754dafd10873ecb99535f3535ee8c76"
}


In [56]:
# Check job status
out_files = check_job(token, apiURL)

Total elapsed time: 0:00:03
REST API JSON response:
{
  "code": 200,
  "state": "FINISHED",
  "message": "The requested job has finished successfully, please go to /retrieve/data/{id} for each output_files.",
  "output_files": [
    {
      "id": "5ec250ecb3e924.28816123",
      "name": "1AKI_npt_PD.xvg",
      "size": 420,
      "mimetype": "text/plain"
    }
  ],
  "expiration": "May 20, 2020 00:00 GMT+0000"
}


In [57]:
# Save generated file to disk
retrieve_data(out_files, apiURL)

In [59]:
import plotly
from plotly import subplots
import plotly.graph_objs as go

# Read pressure and density data from file 
with open(output_npt_pd_xvg,'r') as pd_file:
    x,y,z = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]),float(line.split()[2]))
            for line in pd_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

trace1 = go.Scatter(
    x=x,y=y
)
trace2 = go.Scatter(
    x=x,y=z
)

fig = subplots.make_subplots(rows=1, cols=2, print_grid=False)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig['layout']['xaxis1'].update(title='Time (ps)')
fig['layout']['xaxis2'].update(title='Time (ps)')
fig['layout']['yaxis1'].update(title='Pressure (bar)')
fig['layout']['yaxis2'].update(title='Density (Kg*m^-3)')

fig['layout'].update(title='Pressure and Density during NPT Equilibration')
fig['layout'].update(showlegend=False)

plotly.offline.iplot(fig)

<a id="free"></a>
***
## Free Molecular Dynamics Simulation
Upon completion of the **two equilibration phases (NVT and NPT)**, the system is now well-equilibrated at the desired temperature and pressure. The **position restraints** can now be released. The last step of the **protein** MD setup is a short, **free MD simulation**, to ensure the robustness of the system. 
- [Step 1](#mdStep1): Creating portable binary run file to run a **free MD simulation**.
- [Step 2](#mdStep2): Run short MD simulation of the **protein system**.
- [Step 3](#mdStep3): Checking results for the final step of the setup process, the **free MD run**. Plotting **Root Mean Square deviation (RMSd)** and **Radius of Gyration (Rgyr)** by time during the **free MD run** step. 
***
**BioBB REST API** end points used:
 - [Grompp](https://mmb.irbbarcelona.org/biobb-api/rest/v1/launch/biobb_md/grompp) from **biobb_md.gromacs.grompp**
 - [Mdrun](https://mmb.irbbarcelona.org/biobb-api/rest/v1/launch/biobb_md/mdrun) from **biobb_md.gromacs.mdrun**
 - [GMXRms](https://mmb.irbbarcelona.org/biobb-api/rest/v1/launch/biobb_analysis/gmx_rms) from **biobb_analysis.gromacs.gmx_rms**
 - [GMXRgyr](https://mmb.irbbarcelona.org/biobb-api/rest/v1/launch/biobb_analysis/gmx_rgyr) from **biobb_analysis.gromacs.gmx_rgyr**
***

<a id="mdStep1"></a>
### Step 1: Creating portable binary run file to run a free MD simulation

The **free** type of the **molecular dynamics parameters (mdp) property** contains the main default parameters to run an **free MD simulation** (see [GROMACS mdp options](http://manual.gromacs.org/documentation/2018/user-guide/mdp-options.html)):

-  integrator               = md
-  dt                       = 0.002 (ps)
-  nsteps                   = 50000

In this particular example, the default parameters will be used: **md** integrator algorithm, a **time step** of **2fs**, and a total of **50,000 md steps** (100ps).

*Please note that for the sake of time this tutorial is only running 100ps of free MD, whereas in the [original example](http://www.mdtutorials.com/gmx/lysozyme/08_MD.html) the simulated time was 1ns (1000ps).*

In [60]:
# Grompp: Creating portable binary run file for mdrun

# Create prop dict and inputs/outputs
output_gppmd_tpr = pdbCode + '_gppmd.tpr'
prop = {
    'mdp':{
        'type': 'free',
        'nsteps':'50000',
        #'tc_grps': "DNA Water_and_ions" # NOTE: uncomment this line if working with DNA
    }
}

# Launch bb on REST API
token = launch_job(url = apiURL + 'launch/biobb_md/grompp',
                   config = prop,
                   input_gro_path = output_npt_gro,
                   input_top_zip_path = output_genion_top_zip,
                   output_tpr_path = output_gppmd_tpr)

{
  "code": 303,
  "state": "RUNNING",
  "message": "The requested job has has been successfully launched, please go to /retrieve/status/{token} for checking job status.",
  "token": "14a4d323a635fe233aea43977c00cddb51c895485a8189544b2086c462cbe34a84f9fa7e5f4234999bd535948d77beda9e40b8517904cb2527087749e119917c"
}


In [61]:
# Check job status
out_files = check_job(token, apiURL)

Total elapsed time: 0:00:01
REST API JSON response:
{
  "code": 200,
  "state": "FINISHED",
  "message": "The requested job has finished successfully, please go to /retrieve/data/{id} for each output_files.",
  "output_files": [
    {
      "id": "5ec2511a7871d7.42131369",
      "name": "1AKI_gppmd.tpr",
      "size": 1426804,
      "mimetype": "application/octet-stream"
    }
  ],
  "expiration": "May 20, 2020 00:00 GMT+0000"
}


In [62]:
# Save generated file to disk
retrieve_data(out_files, apiURL)

<a id="mdStep2"></a>
### Step 2: Running short free MD simulation

In [63]:
# Mdrun: Running free dynamics

# Create inputs/outputs
output_md_trr = pdbCode + '_md.trr'
output_md_gro = pdbCode + '_md.gro'
output_md_edr = pdbCode + '_md.edr'
output_md_log = pdbCode + '_md.log'
output_md_cpt = pdbCode + '_md.cpt'

# Launch bb on REST API
token = launch_job(url = apiURL + 'launch/biobb_md/mdrun',
                   input_tpr_path = output_gppmd_tpr,
                   output_trr_path = output_md_trr,
                   output_gro_path = output_md_gro,
                   output_edr_path = output_md_edr,
                   output_log_path = output_md_log,
                   output_cpt_path = output_md_cpt)

{
  "code": 303,
  "state": "RUNNING",
  "message": "The requested job has has been successfully launched, please go to /retrieve/status/{token} for checking job status.",
  "token": "2c78087a1f908d420427d0588c727fcb44943e2f70567cb104beef7b79899e2f6f427ad6910bc02537f6c576a6494e7480060a0146105f5ad36e352bbfdbb175"
}


In [64]:
# Check job status
out_files = check_job(token, apiURL)

Total elapsed time: 0:09:00
REST API JSON response:
{
  "code": 200,
  "state": "FINISHED",
  "message": "The requested job has finished successfully, please go to /retrieve/data/{id} for each output_files.",
  "output_files": [
    {
      "id": "5ec2532db8d800.40685842",
      "name": "1AKI_md.trr",
      "size": 8934024,
      "mimetype": "application/octet-stream"
    },
    {
      "id": "5ec2532e1ea1d9.78681264",
      "name": "1AKI_md.gro",
      "size": 2334738,
      "mimetype": "application/octet-stream"
    },
    {
      "id": "5ec2532e3cd365.11863791",
      "name": "1AKI_md.edr",
      "size": 8208,
      "mimetype": "application/octet-stream"
    },
    {
      "id": "5ec2532e3e54e9.70775617",
      "name": "1AKI_md.log",
      "size": 28151,
      "mimetype": "text/plain"
    },
    {
      "id": "5ec2532e3fc326.00256327",
      "name": "1AKI_md.cpt",
      "size": 814152,
      "mimetype": "application/octet-stream"
    }
  ],
  "expiration": "May 20, 2020 00:00 GMT+00

In [65]:
# Save generated file to disk
retrieve_data(out_files, apiURL)

<a id="mdStep3"></a>
### Step 3: Checking free MD simulation results
Checking results for the final step of the setup process, the **free MD run**. Plotting **Root Mean Square deviation (RMSd)** and **Radius of Gyration (Rgyr)** by time during the **free MD run** step. **RMSd** against the **experimental structure** (input structure of the pipeline) and against the **minimized and equilibrated structure** (output structure of the NPT equilibration step).

In [66]:
# GMXRms: Computing Root Mean Square deviation to analyse structural stability 
#         RMSd against minimized and equilibrated snapshot (backbone atoms)  

# Create prop dict and inputs/outputs
output_rms_first = pdbCode + '_rms_first.xvg'
prop = {
    'selection':  'Backbone',
    #'selection': 'non-Water'
}

# Launch bb on REST API
token = launch_job(url = apiURL + 'launch/biobb_analysis/gmx_rms', 
                   config = prop,
                   input_structure_path = output_gppmd_tpr,
                   input_traj_path = output_md_trr,
                   output_xvg_path = output_rms_first)

{
  "code": 303,
  "state": "RUNNING",
  "message": "The requested job has has been successfully launched, please go to /retrieve/status/{token} for checking job status.",
  "token": "c4ce832243fa6ed7551897d15f9a4157c822771ea159743daad24cce225b15cadbb17379650e9b7c3373918f180cb63cfb84908fee16ef5a33c4772aeccfd5d3"
}


In [67]:
# Check job status
out_files = check_job(token, apiURL)

Total elapsed time: 0:00:04
REST API JSON response:
{
  "code": 200,
  "state": "FINISHED",
  "message": "The requested job has finished successfully, please go to /retrieve/data/{id} for each output_files.",
  "output_files": [
    {
      "id": "5ec253f932ab23.05083090",
      "name": "1AKI_rms_first.xvg",
      "size": 286,
      "mimetype": "text/plain"
    }
  ],
  "expiration": "May 20, 2020 00:00 GMT+0000"
}


In [68]:
# Save generated file to disk
retrieve_data(out_files, apiURL)

In [69]:
# GMXRms: Computing Root Mean Square deviation to analyse structural stability 
#         RMSd against experimental structure (backbone atoms)  

# Create prop dict and inputs/outputs
output_rms_exp = pdbCode + '_rms_exp.xvg'
prop = {
    'selection':  'Backbone',
    #'selection': 'non-Water'
}

# Launch bb on REST API
token = launch_job(url = apiURL + 'launch/biobb_analysis/gmx_rms',
                   config = prop,
                   input_structure_path = output_gppmin_tpr,
                   input_traj_path = output_md_trr,
                   output_xvg_path = output_rms_exp)

{
  "code": 303,
  "state": "RUNNING",
  "message": "The requested job has has been successfully launched, please go to /retrieve/status/{token} for checking job status.",
  "token": "0d35070f491441342f6043a1b300961304664a71448588cf552b81d12270ce2c4c382c32f1f3c75b26ac15f36d237115f829fd59b989cb1175e59fa2d33807f8"
}


In [70]:
# Check job status
out_files = check_job(token, apiURL)

Total elapsed time: 0:00:05
REST API JSON response:
{
  "code": 200,
  "state": "FINISHED",
  "message": "The requested job has finished successfully, please go to /retrieve/data/{id} for each output_files.",
  "output_files": [
    {
      "id": "5ec25407ee4bc3.27703052",
      "name": "1AKI_rms_exp.xvg",
      "size": 286,
      "mimetype": "text/plain"
    }
  ],
  "expiration": "May 20, 2020 00:00 GMT+0000"
}


In [71]:
# Save generated file to disk
retrieve_data(out_files, apiURL)

In [72]:
import plotly
import plotly.graph_objs as go

# Read RMS vs first snapshot data from file 
with open(output_rms_first,'r') as rms_first_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rms_first_file 
            if not line.startswith(("#","@")) 
        ])
    )

# Read RMS vs experimental structure data from file 
with open(output_rms_exp,'r') as rms_exp_file:
    x2,y2 = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rms_exp_file
            if not line.startswith(("#","@")) 
        ])
    )
    
trace1 = go.Scatter(
    x = x,
    y = y,
    name = 'RMSd vs first'
)

trace2 = go.Scatter(
    x = x,
    y = y2,
    name = 'RMSd vs exp'
)

data = [trace1, trace2]

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": data,
    "layout": go.Layout(title="RMSd during free MD Simulation",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "RMSd (nm)")
                       )
}

plotly.offline.iplot(fig)


In [73]:
# GMXRgyr: Computing Radius of Gyration to measure the protein compactness during the free MD simulation 

# Create prop dict and inputs/outputs
output_rgyr = pdbCode + '_rgyr.xvg'
prop = {
    'selection':  'Backbone'
}

# Launch bb on REST API
token = launch_job(url = apiURL + 'launch/biobb_analysis/gmx_rgyr', 
                   config = prop,
                   input_structure_path = output_gppmin_tpr,
                   input_traj_path = output_md_trr,
                   output_xvg_path = output_rgyr)

{
  "code": 303,
  "state": "RUNNING",
  "message": "The requested job has has been successfully launched, please go to /retrieve/status/{token} for checking job status.",
  "token": "63d6343001f20c25e720663c098d176d0c1ef5ab189c753b4fa9a5e413d626a519b11b35993f6a52925cff8aa95d76fc8e77e73763154ba3f86e3942dc801bcd"
}


In [74]:
# Check job status
out_files = check_job(token, apiURL)

Total elapsed time: 0:00:01
REST API JSON response:
{
  "code": 200,
  "state": "FINISHED",
  "message": "The requested job has finished successfully, please go to /retrieve/data/{id} for each output_files.",
  "output_files": [
    {
      "id": "5ec25416e6ad14.33710843",
      "name": "1AKI_rgyr.xvg",
      "size": 649,
      "mimetype": "text/plain"
    }
  ],
  "expiration": "May 20, 2020 00:00 GMT+0000"
}


In [75]:
# Save generated file to disk
retrieve_data(out_files, apiURL)

In [76]:
import plotly
import plotly.graph_objs as go

# Read Rgyr data from file 
with open(output_rgyr,'r') as rgyr_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rgyr_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Radius of Gyration",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "Rgyr (nm)")
                       )
}

plotly.offline.iplot(fig)

<a id="post"></a>
***
## Post-processing and Visualizing resulting 3D trajectory
Post-processing and Visualizing the **protein system** MD setup **resulting trajectory** using **NGL**
- [Step 1](#ppStep1): *Imaging* the resulting trajectory, **stripping out water molecules and ions** and **correcting periodicity issues**.
- [Step 2](#ppStep2): Generating a *dry* structure, **removing water molecules and ions** from the final snapshot of the MD setup pipeline.
- [Step 3](#ppStep3): Visualizing the *imaged* trajectory using the *dry* structure as a **topology**. 
***
**BioBB REST API** end points used:
 - [GMXImage](https://mmb.irbbarcelona.org/biobb-api/rest/v1/launch/biobb_analysis/gmx_image) from **biobb_analysis.gromacs.gmx_image**
 - [GMXTrjConvStr](https://mmb.irbbarcelona.org/biobb-api/rest/v1/launch/biobb_analysis/gmx_trjconv_str) from **biobb_analysis.gromacs.gmx_trjconv_str**
***

<a id="ppStep1"></a>
### Step 1: *Imaging* the resulting trajectory.
Stripping out **water molecules and ions** and **correcting periodicity issues**  

In [77]:
# GMXImage: "Imaging" the resulting trajectory
#           Removing water molecules and ions from the resulting structure

# Create prop dict and inputs/outputs
output_imaged_traj = pdbCode + '_imaged_traj.trr'
prop = {
    'center_selection':  'Protein',
    'output_selection': 'Protein',
    'pbc' : 'mol',
    'center' : True
}

# Launch bb on REST API
token = launch_job(url = apiURL + 'launch/biobb_analysis/gmx_image',
                   config = prop,
                   input_traj_path = output_md_trr,
                   input_top_path = output_gppmd_tpr,
                   output_traj_path = output_imaged_traj)

{
  "code": 303,
  "state": "RUNNING",
  "message": "The requested job has has been successfully launched, please go to /retrieve/status/{token} for checking job status.",
  "token": "3dd0dbd5177027df10be6784f2903da6317aa65c744415d85fdf870ecf5cfa95e1348a156bf2b84c48ad05829bcac0d04c4c541eec2d257476285d67c2dd2ede"
}


In [78]:
# Check job status
out_files = check_job(token, apiURL)

Total elapsed time: 0:00:01
REST API JSON response:
{
  "code": 200,
  "state": "FINISHED",
  "message": "The requested job has finished successfully, please go to /retrieve/data/{id} for each output_files.",
  "output_files": [
    {
      "id": "5ec2542607acb5.09746830",
      "name": "1AKI_imaged_traj.trr",
      "size": 518760,
      "mimetype": "application/octet-stream"
    }
  ],
  "expiration": "May 20, 2020 00:00 GMT+0000"
}


In [79]:
# Save generated file to disk
retrieve_data(out_files, apiURL)

<a id="ppStep2"></a>
### Step 2: Generating the output *dry* structure.
**Removing water molecules and ions** from the resulting structure

In [80]:
# GMXTrjConvStr: Converting and/or manipulating a structure
#                Removing water molecules and ions from the resulting structure
#                The "dry" structure will be used as a topology to visualize 
#                the "imaged dry" trajectory generated in the previous step.

# Create prop dict and inputs/outputs
output_dry_gro = pdbCode + '_md_dry.gro'
prop = {
    'selection':  'Protein'
}

# Launch bb on REST API
token = launch_job(url = apiURL + 'launch/biobb_analysis/gmx_trjconv_str',
                   config = prop,
                   input_structure_path = output_md_gro,
                   input_top_path = output_gppmd_tpr,
                   output_str_path = output_dry_gro)

{
  "code": 303,
  "state": "RUNNING",
  "message": "The requested job has has been successfully launched, please go to /retrieve/status/{token} for checking job status.",
  "token": "0c58c8c26bca4445eab6344c926de43a567e4639b3e00350fed9bbc2f8ad0194bd7f1137f94fa7781736aaf944cf561e1b6be33171bc4d119ce06ce3376ad6a8"
}


In [81]:
# Check job status
out_files = check_job(token, apiURL)

Total elapsed time: 0:00:04
REST API JSON response:
{
  "code": 200,
  "state": "FINISHED",
  "message": "The requested job has finished successfully, please go to /retrieve/data/{id} for each output_files.",
  "output_files": [
    {
      "id": "5ec25434edb4e4.11360538",
      "name": "1AKI_md_dry.gro",
      "size": 135294,
      "mimetype": "application/octet-stream"
    }
  ],
  "expiration": "May 20, 2020 00:00 GMT+0000"
}


In [82]:
# Save generated file to disk
retrieve_data(out_files, apiURL)

<a id="ppStep3"></a>
### Step 3: Visualizing the generated dehydrated trajectory.
Using the **imaged trajectory** (output of the [Post-processing step 1](#ppStep1)) with the **dry structure** (output of the [Post-processing step 2](#ppStep2)) as a topology.

In [83]:
# Show trajectory
view = nglview.show_simpletraj(nglview.SimpletrajTrajectory(output_imaged_traj, output_dry_gro), gui=True)
view

NGLWidget(max_frame=10)

<a id="output"></a>
## Output files

Important **Output files** generated:
 - {{output_md_gro}}: **Final structure** (snapshot) of the MD setup protocol.
 - {{output_md_trr}}: **Final trajectory** of the MD setup protocol.
 - {{output_md_cpt}}: **Final checkpoint file**, with information about the state of the simulation. It can be used to **restart** or **continue** a MD simulation.
 - {{output_gppmd_tpr}}: **Final tpr file**, GROMACS portable binary run input file. This file contains the starting structure of the **MD setup free MD simulation step**, together with the molecular topology and all the simulation parameters. It can be used to **extend** the simulation.
 - {{output_genion_top_zip}}: **Final topology** of the MD system. It is a compressed zip file including a **topology file** (.top) and a set of auxiliar **include topology** files (.itp).

**Analysis** (MD setup check) output files generated:
 - {{output_rms_first}}: **Root Mean Square deviation (RMSd)** against **minimized and equilibrated structure** of the final **free MD run step**.
 - {{output_rms_exp}}: **Root Mean Square deviation (RMSd)** against **experimental structure** of the final **free MD run step**.
 - {{output_rgyr}}: **Radius of Gyration** of the final **free MD run step** of the **setup pipeline**.
 